## Experiments: Comparing RSAST with other methods

In [458]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nir

In [459]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import os
import seaborn as sns
import math
hyperparameter='ACF&PACF: n_random_points=10 nb_inst_per_class=50'

### Reading Datasets

In [460]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]



In [461]:
'''
# Set directory where the csv files are located
df_other_methods=df_other_methods[df_other_methods.method!="STC"]
filepath = os.getcwd()+'/ResultsByClassifier/SAST_TESTFOLDS2.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','STC':'score'})
df['method']="STC"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])
'''

'\n# Set directory where the csv files are located\ndf_other_methods=df_other_methods[df_other_methods.method!="STC"]\nfilepath = os.getcwd()+\'/ResultsByClassifier/SAST_TESTFOLDS2.csv\'\ndf = pd.read_csv(filepath)\ndf = df.rename(columns={\'folds:\':\'dataset\',\'STC\':\'score\'})\ndf[\'method\']="STC"\ndf=df[[\'dataset\',\'score\',\'method\']]\ndf_other_methods=pd.concat([df_other_methods,df])\n'

In [462]:
df_other_methods['method'].unique()

array(['FS', 'ROCKET', 'ELIS++', 'HIVE-COTEv1', 'LS', 'BOSS', 'RISE',
       'SAST', 'TSF', 'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC',
       'ResNet', 'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'],
      dtype=object)

In [463]:
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['SAST','ROCKET','STC','HIVE-COTEv1'])]
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['SAST'])]
df_other_methods=df_other_methods[df_other_methods['method'].isin(['ELIS++','LS','FS','SAST'])]

In [464]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_Extra_1')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)

/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_5890/1115793581.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_5890/1115793581.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of rege

/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_5890/1115793581.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_5890/1115793581.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_5890/1115793581.py:29: FutureWarning: The default value of rege

In [465]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]

df_result=df_result[df_result["classifier_name"]==hyperparameter]

df_result.dataset_name.unique()


array(['MiddlePhalanxOutlineAgeGroup', 'FreezerSmallTrain', 'Wafer',
       'Earthquakes', 'Beef', 'DistalPhalanxOutlineAgeGroup', 'ECG200',
       'ShapeletSim', 'ItalyPowerDemand', 'ToeSegmentation2',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'ToeSegmentation1', 'TwoPatterns', 'WordSynonyms', 'UMD',
       'Strawberry', 'FaceAll', 'GunPointMaleVersusFemale',
       'GunPointOldVersusYoung', 'PowerCons', 'SonyAIBORobotSurface2',
       'BirdChicken', 'Herring', 'CricketY', 'FiftyWords',
       'FreezerRegularTrain', 'Trace', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'BeetleFly', 'FaceFour',
       'ProximalPhalanxTW', 'SwedishLeaf', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'ECG5000', 'GunPoint', 'Meat',
       'SmoothSubspace', 'DiatomSizeReduction', 'CricketZ', 'CricketX',
       'ChlorineConcentration', 'ElectricDevices', 'Car',
       'GunPointAgeSpan', 'ECGFiveDays', 'Chinatown', 'Adiac',
       'DistalPhalanxOutlineCorrect', 'Plane

In [466]:
len(df_result.dataset_name.unique())

66

In [467]:

df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
df_other_methods = df_other_methods[df_other_methods['dataset_name'].isin(df_result.dataset_name.values)]

df_other_methods.columns

Index(['dataset_name', 'accuracy', 'classifier_name'], dtype='object')

In [468]:
df_other_methods

,dataset_name,accuracy,classifier_name
0,ArrowHead,0.594,FS
1,Beef,0.567,FS
2,BeetleFly,0.700,FS
3,BirdChicken,0.750,FS
4,Car,0.750,FS
...,...,...,...
617,TwoPatterns,0.990,SAST
618,UMD,0.980,SAST
619,Wafer,1.000,SAST
620,Wine,0.850,SAST


In [469]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [470]:
concatenated_df['classifier_name'].unique()

array(['FS', 'ELIS++', 'LS', 'SAST',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=50'], dtype=object)

In [471]:
len(concatenated_df.dataset_name.unique())

66

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [472]:
max_bx=max(df_result.accuracy)
min_bx=min(df_result.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=df_result, y='dataset_name', x='accuracy')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
#plt.tight_layout()
# save plot
plt.savefig('images_boxplot_acc/boxplot_acc_per_ds.png')

/tmp/ipykernel_5890/853197985.py:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')


In [473]:
filter_param=concatenated_df[concatenated_df.classifier_name==hyperparameter]

In [474]:
stats = filter_param.groupby(['dataset_name'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []
ci95_hi_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))
    ci95_hi_lo.append(1.96*s/math.sqrt(c))

#stats['ci95_hi'] = ci95_hi
#stats['ci95_lo'] = ci95_lo
stats['ci95_interval'] = ci95_hi_lo
print(stats.round(2))

              mean  count   std  ci95_interval
dataset_name                                  
Adiac         0.58      5  0.01           0.01
ArrowHead     0.79      5  0.01           0.01
BME           0.83      5  0.01           0.00
Beef          0.68      5  0.02           0.02
BeetleFly     0.78      5  0.03           0.02
...            ...    ...   ...            ...
TwoPatterns   1.00      5  0.00           0.00
UMD           0.98      5  0.00           0.00
Wafer         1.00      5  0.00           0.00
Wine          0.79      5  0.02           0.02
WordSynonyms  0.68      5  0.00           0.00

[66 rows x 4 columns]


In [475]:
# create a pivot table with the mean of score by hyperparameter
score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=score_by_method.transpose()[[hyperparameter,"SAST"]].round(2)
t_score_by_method=score_by_method.transpose().round(2)
t_score_by_method.iloc[0:25,:]


classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=50,ELIS++,FS,LS,SAST
dataset_name,,,,,
Adiac,0.58,NaN,NaN,NaN,0.68
ArrowHead,0.79,0.82,0.59,0.85,0.77
BME,0.83,NaN,NaN,NaN,0.87
Beef,0.68,0.67,0.57,0.73,0.80
BeetleFly,0.78,1.00,0.70,0.75,0.80
BirdChicken,0.85,1.00,0.75,0.80,0.76
CBF,0.99,0.95,0.94,0.99,0.98
Car,0.84,0.65,0.75,0.77,0.88
Chinatown,0.98,NaN,NaN,NaN,0.96


In [476]:
t_score_by_method.iloc[25:49,:]

classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=50,ELIS++,FS,LS,SAST
dataset_name,,,,,
FaceFour,1.00,0.96,0.91,0.96,1.00
FacesUCR,0.96,NaN,NaN,NaN,0.95
FiftyWords,0.77,NaN,NaN,NaN,0.77
FreezerRegularTrain,0.98,NaN,NaN,NaN,0.98
FreezerSmallTrain,0.85,NaN,NaN,NaN,0.73
Fungi,1.00,NaN,NaN,NaN,0.90
GunPoint,0.98,0.99,0.95,0.99,0.97
GunPointAgeSpan,0.97,NaN,NaN,NaN,0.97
GunPointMaleVersusFemale,0.99,NaN,NaN,NaN,0.99


In [477]:
t_score_by_method.iloc[49:75,:]

classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=50,ELIS++,FS,LS,SAST
dataset_name,,,,,
ShapeletSim,0.88,1.00,1.00,0.95,0.96
SmoothSubspace,0.94,NaN,NaN,NaN,0.91
SonyAIBORobotSurface1,0.81,0.92,0.69,0.81,0.76
SonyAIBORobotSurface2,0.91,NaN,NaN,NaN,0.85
Strawberry,0.97,NaN,NaN,NaN,0.97
SwedishLeaf,0.90,NaN,NaN,NaN,0.88
Symbols,0.94,0.93,0.93,0.93,0.95
SyntheticControl,0.97,0.98,0.91,1.00,0.98
ToeSegmentation1,0.89,0.97,0.96,0.93,0.88


In [478]:
t_score_by_method.describe().round(2)

classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=50,ELIS++,FS,LS,SAST
count,66.00,33.00,33.00,33.00,66.00
mean,0.85,0.86,0.78,0.82,0.85
std,0.14,0.14,0.15,0.15,0.12
min,0.41,0.58,0.53,0.50,0.53
25%,0.78,0.77,0.69,0.73,0.76
50%,0.88,0.92,0.75,0.81,0.87
75%,0.97,0.98,0.92,0.95,0.96
max,1.00,1.00,1.00,1.00,1.00


In [479]:
score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=score_by_method.transpose()[[hyperparameter,"SAST"]].round(2)
total_wins=len(t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))])
print("winning RSAST: "+str(total_wins))

winning RSAST: 29


In [480]:
total_loss=len(t_score_by_method[0>(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))])
print("losse RSAST: "+str(total_loss))

losse RSAST: 21


In [481]:
total_draws=len(t_score_by_method[0==(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))])
print("draws: "+str(total_draws))

draws: 16


In [482]:
#wins of RSAST
t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))].index

Index(['ArrowHead', 'BirdChicken', 'CBF', 'Chinatown', 'ChlorineConcentration',
       'CricketX', 'CricketY', 'CricketZ', 'ECG200', 'Earthquakes', 'FacesUCR',
       'FreezerSmallTrain', 'Fungi', 'GunPoint', 'GunPointOldVersusYoung',
       'Ham', 'Herring', 'Meat', 'MoteStrain', 'PhalangesOutlinesCorrect',
       'ProximalPhalanxTW', 'SmoothSubspace', 'SonyAIBORobotSurface1',
       'SonyAIBORobotSurface2', 'SwedishLeaf', 'ToeSegmentation1',
       'ToeSegmentation2', 'TwoLeadECG', 'TwoPatterns'],
      dtype='object', name='dataset_name')

In [483]:
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'], aggfunc='var').round(4)

,accuracy
classifier_name,
ACF&PACF: n_random_points=10 nb_inst_per_class=50,0.0189
ELIS++,0.0206
FS,0.0222
LS,0.0223
SAST,0.0154


In [484]:
for method in t_score_by_method.columns.drop(hyperparameter):
    y=t_score_by_method[hyperparameter]
    x=t_score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_ylabel('RSAST')
    ax.set_xlabel(method)
    ax.set_title('Accuracy')


    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('images_one_vs_one_comparison/RSASTvs'+method+'.png')


/tmp/ipykernel_5890/3124073668.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [485]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=50
dataset_name,
Adiac,0.474674
ArrowHead,0.095713
BME,0.014643
Beef,0.450049
BeetleFly,0.402148
...,...
TwoPatterns,0.183257
UMD,0.022466
Wafer,0.099204


In [486]:
# create a pivot table with the variation of score by hyperparameter
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='var')


dataset_name,Adiac,ArrowHead,BME,Beef,BeetleFly,BirdChicken,CBF,Car,Chinatown,ChlorineConcentration,...,SyntheticControl,ToeSegmentation1,ToeSegmentation2,Trace,TwoLeadECG,TwoPatterns,UMD,Wafer,Wine,WordSynonyms
classifier_name,,,,,,,,,,,,,,,,,,,,,
ACF&PACF: n_random_points=10 nb_inst_per_class=50,0.000181,0.000137,0.000031,0.000333,0.00075,0.0,0.000001,0.000083,0.000019,0.000043,...,0.000011,0.000025,0.000089,0.0,0.000013,2.062500e-07,0.0,0.0,0.000412,0.000006


#### Generate CD Diagram comparison

In [487]:
pv_all=pd.pivot_table(concatenated_df,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
concatenated_df=concatenated_df[concatenated_df.dataset_name.isin(ds_complete)]

In [488]:
concatenated_df=concatenated_df[concatenated_df.dataset_name!="Fungi"]
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'], aggfunc='count')

,accuracy
classifier_name,
ACF&PACF: n_random_points=10 nb_inst_per_class=50,33
ELIS++,33
FS,33
LS,33
SAST,33


In [489]:
concatenated_df.classifier_name.unique()

array(['FS', 'ELIS++', 'LS', 'SAST',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=50'], dtype=object)

In [490]:

from cd_function import *
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, title="Overall Comparison Other Methods", fname='images_cd_diagram/cd-diagram_overall_comparison_other_method.png')


{0: 'FS', 1: 'LS', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=50', 3: 'SAST', 4: 'ELIS++'}
[1, 2, 3, 4]


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [491]:
df_perf["classifier_name"].unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=50', 'ELIS++',
       'FS', 'LS', 'SAST'], dtype=object)

In [492]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
FS,4.303030
LS,2.833333
ACF&PACF: n_random_points=10 nb_inst_per_class=50,2.803030
SAST,2.787879
ELIS++,2.272727
